# Cypher Query Examples

Quick reference notebook with common Cypher query patterns and examples.

## Setup
Run the cell below to initialize the environment.


In [ ]:
# Basic setup
from rich.console import Console
from rich.table import Table

from genai_graph.core.graph_backend import create_backend_from_config
from genai_graph.core.text2cypher import query_kg

console = Console()
backend = create_backend_from_config("default")


def run_query(query: str, title: str = "Results"):
    """Simple query execution function."""
    try:
        result = backend.execute(query)
        df = result.get_as_df()

        table = Table(title=f"{title} ({len(df)} rows)")
        for col in df.columns:
            table.add_column(str(col), style="cyan")

        for _, row in df.iterrows():
            table.add_row(*[str(val) for val in row])

        console.print(table)
        return df
    except Exception as e:
        console.print(f"[red]Error: {e}[/red]")
        return None


print("✅ Setup complete. Ready to run queries!")

2025-11-28 22:30:42.048 | INFO     | genai_graph.core.graph_registry:model_post_init:62 - import genai_graph.ekg.schema.rainbow_review:ReviewedOpportunitySubgraph
2025-11-28 22:30:42.289 | INFO     | genai_graph.core.graph_registry:model_post_init:62 - import genai_graph.ekg.schema.architecture_doc:ArchitectureDocumentSubgraph


✅ Setup complete. Ready to run queries!


## Basic Queries


In [2]:
# Count nodes by type
query = "MATCH (n) RETURN labels(n)[0] as type, count(n) as count ORDER BY count DESC"
run_query(query, "Node Counts by Type")

 Node Counts by 
 Type (1 rows)  
┏━━━━━━┳━━━━━━━┓
┃ type ┃ count ┃
┡━━━━━━╇━━━━━━━┩
│      │ 56    │
└──────┴───────┘

,type,count
0,,56


In [3]:
# List relationship types
query = "MATCH ()-[r]->() RETURN type(r) as relationship, count(*) as count ORDER BY count DESC"
run_query(query, "Relationship Types")

Error: Catalog exception: function TYPE does not exist.

In [4]:
# Sample opportunities
query = "MATCH (n:ReviewedOpportunity) RETURN n.name, n.document_date ORDER BY n.document_date DESC LIMIT 5"
run_query(query, "Sample Opportunities")

Error: Binder exception: Cannot find property name for n.

## Relationship Queries


In [ ]:
# Opportunities with competitors
query = """
MATCH (ro:ReviewedOpportunity)-[:HAS_COMPETITOR]->(c:Competitor)
RETURN ro.name as opportunity, c.name as competitor
ORDER BY ro.name
LIMIT 10
"""
run_query(query, "Opportunities with Competitors")

In [ ]:
# Opportunities with customers
query = """
MATCH (ro:ReviewedOpportunity)-[:HAS_CUSTOMER]->(cust:Customer)
RETURN ro.name as opportunity, cust.name as customer
ORDER BY ro.name
LIMIT 10
"""
run_query(query, "Opportunities with Customers")

In [ ]:
# Count competitors per opportunity
query = """
MATCH (ro:ReviewedOpportunity)-[:HAS_COMPETITOR]->(c:Competitor)
RETURN ro.name as opportunity, count(c) as competitor_count
ORDER BY competitor_count DESC
LIMIT 10
"""
run_query(query, "Competitors per Opportunity")

## Statistical Queries


In [ ]:
# Financial statistics
query = """
MATCH (n:ReviewedOpportunity)
RETURN 
  count(n) as total_count,
  avg(n.financials.tcv) as avg_tcv,
  max(n.financials.tcv) as max_tcv,
  min(n.financials.tcv) as min_tcv
"""
run_query(query, "Financial Statistics")

In [ ]:
# Opportunities by year
query = """
MATCH (n:ReviewedOpportunity)
RETURN substring(n.document_date, 0, 4) as year, count(n) as count
ORDER BY year DESC
"""
run_query(query, "Opportunities by Year")

## Text-to-Cypher Examples


In [ ]:
# Simple text query
query = "List all opportunities with their competitors"
try:
    result = query_kg(query, subgraphs=[])
    print(f"Results: {len(result)} rows")
    print(result.head())
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Statistical text query
query = "What is the average TCV of opportunities?"
try:
    result = query_kg(query, subgraphs=[])
    print(result)
except Exception as e:
    print(f"Error: {e}")

## Quick Reference

### Common Cypher Patterns

**Count nodes:**
```
MATCH (n:Label) RETURN count(n)
```

**Find with property:**
```
MATCH (n:Label) WHERE n.property = 'value' RETURN n
```

**Relationships:**
```
MATCH (a)-[:REL_TYPE]->(b) RETURN a, b
```

**Aggregation:**
```
MATCH (n) RETURN n.category, avg(n.value) GROUP BY n.category
```

**Date filtering:**
```
MATCH (n) WHERE date(n.date) > date('2020-01-01') RETURN n
```

### Text-to-Cypher Tips

- Be specific about what you want
- Mention node types when possible
- Include date ranges and filters
- Ask for statistics explicitly
- Reference relationships naturally
